In [13]:
import numpy as np # linear algebra
import pandas as pd

In [14]:
import re
import string   
from wordcloud import WordCloud
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk import pos_tag, ne_chunk
from nltk.chunk import tree2conlltags

import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CHETAN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CHETAN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\CHETAN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\CHETAN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\CHETAN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping chunkers\maxent_ne_chunker.zip.
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\CHETAN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


In [16]:
labels = ["Twitter_ID","Entity","Sentiment","Tweet_content"]
data = pd.read_csv("C:\\Users\\CHETAN\\OneDrive\\Desktop\\PES1UG20CS109\\MI\\Project\\twitter_training.csv",
                   names=labels,header=None)

In [17]:
data.head()

,Id,Entity,Target,Text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [37]:
training_df = data[["Tweet_content","Sentiment"]]
training_df= training_df.drop_duplicates()

In [20]:
sentiment = []

for i in training_df["Sentiment"]:
    if i == "Positive":
        sentiment.append(1)
    elif (i == "Irrelevant") or (i == "Neutral"):
        sentiment.append(0)
    else:
        sentiment.append(-1)
training_df["Sentiment"] = sentiment

In [36]:
stop_words = set(stopwords.words("english"))
training_df["Tweet_content"] = training_df["Tweet_content"].str.replace("\d","")

In [23]:
def cleaner(data):

    tokens = word_tokenize(str(data).replace("'", "").lower()) 
    without_punc = [w for w in tokens if w.isalpha()]
    without_sw = [t for t in without_punc if t not in stop_words]
    Tweet_content_len = [WordNetLemmatizer().lemmatize(t) for t in without_sw]
    Tweet_content_cleaned = [PorterStemmer().stem(w) for w in Tweet_content_len]
    return " ".join(Tweet_content_cleaned)

In [24]:
training_df["Tweet_content"] = training_df["Tweet_content"].apply(cleaner)
training_df["Tweet_content"].head()

0     im get borderland murder
1             come border kill
2       im get borderland kill
3    im come borderland murder
4     im get borderland murder
Name: Text, dtype: object

In [25]:
training_df["Tweet_content"]=training_df["Tweet_content"].str.replace("im","")
training_df["Tweet_content"].head()

0      get borderland murder
1           come border kill
2        get borderland kill
3     come borderland murder
4      get borderland murder
Name: Text, dtype: object

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [27]:
X = training_df["Tweet_content"]
y = training_df["Sentiment"]

In [28]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.30,random_state= 42,stratify = y)

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

In [30]:
vt = CountVectorizer(analyzer="word")
X_train_count = vt.fit_transform(X_train)
X_test_count = vt.transform(X_test)

In [33]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn_model = knn.fit(X_train_count,y_train)

In [39]:
knn_pred = knn_model.predict(X_test_count)


In [38]:
print("X Test")
print(classification_report(y_test,knn_pred))


X Test
              precision    recall  f1-score   support

          -1       0.83      0.86      0.84      6371
           0       0.86      0.82      0.84      8819
           1       0.81      0.82      0.81      5742

    accuracy                           0.83     20932
   macro avg       0.83      0.83      0.83     20932
weighted avg       0.83      0.83      0.83     20932

